<a href="https://colab.research.google.com/github/HannaPylieva-GBI/classification/blob/main/lightgbm_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 88.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import lightgbm as lgbm

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

from tqdm import tqdm #progress bar

# from spellchecker import SpellChecker

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 282
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_24234/3964101490.py", line 9, in <module>
    import tensorflow_hub as hub
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_hub/__init__.py", line 29, in <module>
    from tensorflow_hub.estimator import LatestModuleExporter
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_hub/estimator.py", line 63, in <module>
    class LatestModuleExporter(tf_v1.estimator.Exporter):
AttributeError: module 'tensorflow_hub.tf_v1' has no attribute 'estimator'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2098, in showtraceback
    stb

In [1]:
train_path = 'gs://gbi_ml/classification_hackathon/bbby_train_new.csv'
test_path = 'gs://gbi_ml/classification_hackathon/bbby_test_new.csv'
# train_path = 'gs://gbi_ml/classification_hackathon/bbby_train_30_tokenization.pkl'
# test_path = 'gs://gbi_ml/classification_hackathon/bbby_test_tokenization.pkl'

In [ ]:
!pip install gcsfs

In [ ]:
!gcloud auth login

In [8]:
!gcloud config set project 'groupby-development'

Updated property [core/project].


In [9]:
from google.colab import auth
auth.authenticate_user()

In [10]:
df = pd.read_csv(train_path)

In [2]:
test_df = pd.read_csv(test_path)

ERROR! Session/line number was not unique in database. History logging moved to new session 278


In [12]:
df.shape, test_df.shape

((308284, 12), (19481, 8))

In [21]:
df.columns

Index(['bucket_id', 'external_id', 'customer_id', 'bucket_name', 'product_id',
       'product_name', 'description', 'image_url', 'len_desc', 'len_name',
       'raw_product_name', 'raw_product_description'],
      dtype='object')

In [3]:
# cats = sorted(df.bucket_name.unique())
# id2label = {id_:name_ for id_, name_ in enumerate(cats)}
# label2id = {name_: id_ for id_, name_ in enumerate(cats)}

ERROR! Session/line number was not unique in database. History logging moved to new session 279


In [4]:
import pickle 
with open('id2label.pickle', 'rb') as handle:
    id2label = pickle.load(handle)
    
with open('label2id.pickle', 'rb') as handle:
    label2id = pickle.load(handle)

ERROR! Session/line number was not unique in database. History logging moved to new session 280


In [5]:
from collections import defaultdict
  
def def_value():
    return -1
      
# Defining the dict
d = defaultdict(int, label2id)

In [6]:
# df['label'] = df.bucket_name.apply(lambda x: label2id[x])
test_df['label'] = test_df['Manual Classification Bucket'].apply(lambda x: d[x])

ERROR! Session/line number was not unique in database. History logging moved to new session 281


In [15]:
from tqdm import tqdm
import tensorflow as tf
import tensorflow_hub as hub


In [ ]:
# SENTENCES EMBEDDING

# Load encoder from Tensorflow for LGBM model
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_24234/3810780756.py", line 4, in <module>
    use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
NameError: name 'hub' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2098, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  F

In [ ]:
# Embed sentences for the training set
X_train = []
for r in tqdm(df.raw_product_description.values):
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_train.append(review_emb)

X_train = np.array(X_train)

In [26]:
y_train = df.label.values

# Embed sentences for the validation set
Val_test = []
for r in tqdm(test_df.raw_product_description.values):
    emb = use(r)
    review_emb = tf.reshape(emb, [-1]).numpy()
    Val_test.append(review_emb)

100%|██████████| 19481/19481 [08:23<00:00, 38.72it/s]


In [27]:
import gcsfs
import pickle 

base_dir = 'gbi_ml/classification_hackathon/'
fs = gcsfs.GCSFileSystem()

with fs.open(base_dir+'X_train.pickle', 'wb') as handle:
    pickle.dump(X_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

with fs.open(base_dir+'X_test.pickle', 'wb') as handle:
    pickle.dump(Val_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
import lightgbm as lgbm

In [43]:
# LGBM

# Split data into train and testing data
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state = 42, stratify=y_train)

# Get the train and test data for the training sequence
train_data = lgbm.Dataset(X_train, label=y_train)
test_data = lgbm.Dataset(X_test, label=y_test)

In [45]:
y_test[:5]

array([601, 447, 394, 303, 665])

In [1]:
# Parameters we'll use for the prediction
parameters = {
    # 'application': 'binary',
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'boosting': 'gbdt',
    # 'num_leaves': 31,
    # 'feature_fraction': 0.5,
    # 'bagging_fraction': 0.5,
    # 'bagging_freq': 20,
    # 'learning_rate': 0.05,
    # 'verbose': 0,
    # 'max_depth': 10,
    # 'device': 'gpu',
    'num_classes': len(id2label),
    'min_data_in_leaf':300
}

# Train the classifier
# model = lgbm.LGBMClassifier()
# model = lgbm.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=- 1, learning_rate=0.1, n_estimators=300, device = "gpu")
classifier = lgbm.train(parameters,
                       train_data,
                       valid_sets=[train_data, test_data],
                      #  num_boost_round=5000,
                       early_stopping_rounds=10)


# PREDICTION
val_pred = classifier.predict(Val_test)

# # Submission file
# submission_df['target'] = val_pred.round().astype(int)
# submission_df.to_csv('submission_lgbm.csv', index=False)

NameError: ignored